In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.task import Console, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models import OpenAIChatCompletionClient
from AzModelClient import create_azure_openai_client

import random

In [ ]:
# 1. tools / Skills
# Define a tool that returns the weather in a city
async def get_weather(city: str) -> str:
    temperature = random.randint(-30, 50)  # Generate a random temperature between -30 and 50 degrees
    weather_conditions = ["Sunny", "Cloudy", "Raining"]
    condition = random.choice(weather_conditions)  # Randomly choose a weather condition
    return f"The weather in {city} is {temperature} degrees and {condition}."

In [ ]:
# 2. Define the Agent.

async def main( promp) -> None:
    # Define an agent
    weather_agent = AssistantAgent(
        name="weather_agent",
        model_client= create_azure_openai_client(),
        #What are actions (skills) this agent can perform
        tools=[get_weather], 
        #Help the planner to engage this agent to solve a complext task
        description="I can provide you with the current weather in a city.", 
        #System message provide the agent the guide on how to work, samples,limits,constrains and all the Prompt enginer tecnics 
        system_message= """You are a helpful assistant. 
        Your task is to provide the user with the current weather in a city and write a poem about the city and the weather.
        your response should end with the word 'TERMINATE'
        """
    )

    # Define termination condition
    termination = TextMentionTermination("TERMINATE")

    # Define a team
    agent_team = RoundRobinGroupChat([weather_agent], termination_condition=termination)

    # Run the team and stream messages to the console
    stream = agent_team.run_stream(task=promp)
    await Console(stream)

In [ ]:
TheTaskToComplete = "What is the weather in Boston?"
# 3. Run the agent
await main(TheTaskToComplete)